<a href="https://colab.research.google.com/github/Propa-Punam/Wifi-RSS-Crowdsensing/blob/main/data%20preprocessing/final_generate_the_filtered_dataset_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import required libraries
import pandas as pd
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define paths for input and output files in Google Drive
input_path_entries = '/content/drive/My Drive/entries.csv'
input_path_scans = '/content/drive/My Drive/scan_lists.csv'
output_path_filtered = '/content/drive/My Drive/filtered_wifi_data_203.csv'
output_path_unique_macs = '/content/drive/My Drive/unique_mac_addresses_203.csv'
output_path_strengths = '/content/drive/My Drive/strength_203.csv'

# Read the CSV files
entries_df = pd.read_csv(input_path_entries)
scans_df = pd.read_csv(input_path_scans)

# List of SSIDs to filter
target_ssids = [
    "CSE-206", "CSE-104", "CSE-202", "CSE-205",
    "CSE-304", "CSE-204", "CSE-303", "DataLab@BUET",
    "CSE-214", "CSE-G04", "CSE-401", "CSE-306"
]

# Provided list of unique MAC addresses and SSIDs
unique_mac_ssid_list = [
    {"MacAddress": "bc:22:28:21:55:6e", "SSID": "CSE-104"},
    {"MacAddress": "bc:22:28:21:55:6b", "SSID": "CSE-104"},
    {"MacAddress": "00:22:b0:05:f3:bc", "SSID": "CSE-204"},
    {"MacAddress": "bc:22:28:21:09:24", "SSID": "CSE-205"},
    {"MacAddress": "bc:22:28:21:09:21", "SSID": "CSE-205"},
    {"MacAddress": "bc:22:28:21:09:ba", "SSID": "CSE-206"},
    {"MacAddress": "bc:22:28:21:70:62", "SSID": "CSE-214"},
    {"MacAddress": "60:63:4c:31:37:40", "SSID": "CSE-303"},
    {"MacAddress": "18:0f:76:c3:06:8d", "SSID": "CSE-304"},
    {"MacAddress": "60:63:4c:31:37:60", "SSID": "CSE-306"},
    {"MacAddress": "00:1e:2a:ba:c8:20", "SSID": "CSE-401"},
    {"MacAddress": "10:62:eb:77:b7:42", "SSID": "CSE-G04"},
    {"MacAddress": "30:b5:c2:ea:5b:44", "SSID": "DataLab@BUET"}
]

# Create lists to store the filtered data and unique MAC addresses
filtered_data = []
mac_ssid_pairs = set()  # Using a set to store unique MAC-SSID pairs

# Iterate through each entry in entries.csv
for _, entry_row in entries_df.iterrows():
    entry_id = entry_row['entryId']
    student_id = entry_row['StudentID']

    # Get all scans for this entry_id
    entry_scans = scans_df[scans_df['entryId'] == entry_id]

    # Iterate through each scan group (0 to 19)
    for scan_group in range(20):
        group_scans = entry_scans[entry_scans['scanGroupIndex'] == scan_group]

        # Filter for target SSIDs
        for _, scan_row in group_scans.iterrows():
            ssid = scan_row['SSID']
            if ssid in target_ssids:
                # Add to filtered data (for first output)
                filtered_data.append({
                    'entryId': entry_id,
                    'StudentID': student_id,
                    'scanGroupIndex': scan_group,
                    'MacAddress': scan_row['MacAddress'],
                    'SSID': ssid,
                    'Strength': scan_row['Strength']
                })

                # Add to unique MAC-SSID pairs
                mac_ssid_pairs.add((scan_row['MacAddress'], ssid))

# Convert filtered data to DataFrame (first output)
output_df_filtered = pd.DataFrame(filtered_data)
output_df_filtered = output_df_filtered.sort_values(['entryId', 'scanGroupIndex'])

# Convert unique MAC-SSID pairs to DataFrame (second output)
unique_macs_data = [{'MacAddress': mac, 'SSID': ssid} for mac, ssid in mac_ssid_pairs]
output_df_unique_macs = pd.DataFrame(unique_macs_data)
output_df_unique_macs = output_df_unique_macs.sort_values('SSID')

# Create a DataFrame for all possible combinations of entryId, StudentID, MacAddress, and SSID
# First, get all unique entryId and StudentID pairs
entry_student_pairs = entries_df[['entryId', 'StudentID']].drop_duplicates()

# Create a DataFrame from the unique MAC-SSID list
mac_ssid_df = pd.DataFrame(unique_mac_ssid_list)

# Create a Cartesian product of entryId-StudentID pairs with MacAddress-SSID pairs
all_combinations = (
    entry_student_pairs.assign(key=1)
    .merge(mac_ssid_df.assign(key=1), on='key')
    .drop('key', axis=1)
)

# Pivot the filtered data to have a column for each scanGroupIndex's strength
pivot_df = output_df_filtered.pivot_table(
    index=['entryId', 'StudentID', 'MacAddress', 'SSID'],
    columns='scanGroupIndex',
    values='Strength',
    aggfunc='first'  # Take the first value if there are duplicates
)

# Reset index to make entryId, StudentID, MacAddress, and SSID regular columns
pivot_df = pivot_df.reset_index()

# Rename the columns to indicate they are strengths for each scan group
pivot_df.columns = ['entryId', 'StudentID', 'MacAddress', 'SSID'] + [f'Strength_scan_{i}' for i in range(20)]

# Merge with all_combinations to ensure all MAC addresses are present for each entryId
final_df = all_combinations.merge(pivot_df, on=['entryId', 'StudentID', 'MacAddress', 'SSID'], how='left')

# Identify rows where all strength values are NaN
strength_columns = [f'Strength_scan_{i}' for i in range(20)]
all_nan_mask = final_df[strength_columns].isna().all(axis=1)

# Assign -100 to all strength columns only for rows where all strengths are NaN
final_df.loc[all_nan_mask, strength_columns] = -100

# Add a column for the count of recorded strengths (excluding -100)
final_df['Strength_Count'] = final_df[strength_columns].apply(
    lambda row: sum(1 for val in row if pd.notna(val) and val != -100), axis=1
)

# Set Strength_Count to 20 for rows where all strengths are -100 (i.e., originally all NaN)
final_df.loc[all_nan_mask, 'Strength_Count'] = 20

# Save all DataFrames to CSV files in Google Drive
output_df_filtered.to_csv(output_path_filtered, index=False)
output_df_unique_macs.to_csv(output_path_unique_macs, index=False)
final_df.to_csv(output_path_strengths, index=False)

# Print summary information
print(f"Filtered data has been saved to {output_path_filtered}")
print(f"Total records in filtered data: {len(output_df_filtered)}")
print(f"\nUnique MAC addresses have been saved to {output_path_unique_macs}")
print(f"Total unique MAC-SSID pairs: {len(output_df_unique_macs)}")
print(f"\nStrengths per scan group have been saved to {output_path_strengths}")
print(f"Total records in strengths data: {len(final_df)}")

# Display the first few rows of all outputs
print("\nFirst few rows of filtered data:")
print(output_df_filtered.head())
print("\nFirst few rows of unique MAC addresses:")
print(output_df_unique_macs.head())
print("\nFirst few rows of strengths per scan group:")
print(final_df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Filtered data has been saved to /content/drive/My Drive/filtered_wifi_data_203.csv
Total records in filtered data: 1930

Unique MAC addresses have been saved to /content/drive/My Drive/unique_mac_addresses_203.csv
Total unique MAC-SSID pairs: 13

Strengths per scan group have been saved to /content/drive/My Drive/strength_203.csv
Total records in strengths data: 325

First few rows of filtered data:
   entryId  StudentID  scanGroupIndex         MacAddress          SSID  \
0        1    2005045               0  bc:22:28:21:09:24       CSE-205   
1        1    2005045               0  bc:22:28:21:09:21       CSE-205   
2        1    2005045               0  30:b5:c2:ea:5b:44  DataLab@BUET   
3        1    2005045               0  60:63:4c:31:37:40       CSE-303   
4        1    2005045               1  bc:22:28:21:09:24       CSE-205   

   Strength  
0       -